[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/SVD.ipynb)

# 특잇값 분해(Singular Value Decomposition, SVD)

In [1]:
# Colab용 notebook입니다. 이 notebook 한 장에서 여러 데이터의 다운로드부터, 추천까지 완결하도록 되어 있습니다(예측 평가는 미포함)
# MovieLens 데이터를 아직 다운로드 하지 않았다면, 이 셀을 실행해서 다운로드합니다.
# MovieLens 데이터 분석은 data_download.ipynb를 참조합니다.

# 데이터 다운로드와 압축 풀기
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

--2022-12-27 05:43:01--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘../data/ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  65.4MB/s    in 1.0s    

2022-12-27 05:43:02 (65.4 MB/s) - ‘../data/ml-10m.zip’ saved [65566137/65566137]

Archive:  ../data/ml-10m.zip
   creating: ../data/ml-10M100K/
  inflating: ../data/ml-10M100K/allbut.pl  
  inflating: ../data/ml-10M100K/movies.dat  
  inflating: ../data/ml-10M100K/ratings.dat  
  inflating: ../data/ml-10M100K/README.html  
  inflating: ../data/ml-10M100K/split_ratings.sh  
  inflating: ../data/ml-10M100K/tags.dat  


In [2]:
# Movielens 데이터 로딩(데이터량이 많으므로, 로딩에 시간이 걸릴 수 있습니다)
import pandas as pd

# movieID와 제목만 사용
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genre를 list 형식으로 저장한다
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# 사용자가 부여한 영화의 태그 정보를 로딩한다
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tag를 소문자로 바꾼다
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tag를 영화별로 list 형식으로 저장한다
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# 태그 정보를 결합한다
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 평갓값 데이터만 로딩한다
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# 데이터량이 많으므로 사용자수를 1000으로 줄여서 시험해본다
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 영화 데이터와 평가 데이터를 결합한다
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 학습용과 데이터용으로 데이터를 나눈다
# 각 사용자의 최근 5건의 영화를 평가용으로 사용하고, 나머지는 학습용으로 사용한다
# 우선, 각 사용자가 평가한 영화의 순서를 계산한다
# 최근 부여한 영화부터 순서를 부여한다(1에서 시작)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
# 결손값을 채우는 방법
fillna_with_zero = True
# 인자 수
factors = 5

In [4]:
# 평갓값을 사용자 x 영화 행렬로 변환한다. 결손값은 평균값 또는 0으로 채운다.
user_movie_matrix = movielens_train.pivot(index="user_id", columns="movie_id", values="rating")
user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))
if fillna_with_zero:
    matrix = user_movie_matrix.fillna(0).to_numpy()
else:
    matrix = user_movie_matrix.fillna(movielens_train.rating.mean()).to_numpy()


In [5]:
# 희소 정보
user_num = len(user_movie_matrix.index)
item_num = len(user_movie_matrix.columns)
non_null_num = user_num*item_num - user_movie_matrix.isnull().sum().sum()
non_null_ratio = non_null_num / (user_num*item_num)

print(f'ユーザー数={user_num}, アイテム数={item_num}, 密度={non_null_ratio:.2f}')

ユーザー数=1000, アイテム数=6673, 密度=0.02


In [6]:
from scipy.sparse.linalg import svds
import numpy as np

# 인자 수 k로 특이점 분석을 수행한다
P, S, Qt = svds(matrix, k=factors)

In [7]:
# 예측 평갓값 행렬
pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)

print(f"P: {P.shape}, S: {S.shape}, Qt: {Qt.shape}, pred_matrix: {pred_matrix.shape}")

P: (1000, 5), S: (5,), Qt: (5, 6673), pred_matrix: (1000, 6673)


In [8]:
# 학습용에 나타나지 않은 사용자와 영화 예측 평갓값은 평균 평갓값으로 한다
average_score = movielens_train.rating.mean()
movie_rating_predict = movielens_test.copy()
pred_results = []
for i, row in movielens_test.iterrows():
    user_id = row["user_id"]
    if user_id not in user_id2index or row["movie_id"] not in movie_id2index:
        pred_results.append(average_score)
        continue
    user_index = user_id2index[row["user_id"]]
    movie_index = movie_id2index[row["movie_id"]]
    pred_score = pred_matrix[user_index, movie_index]
    pred_results.append(pred_score)
movie_rating_predict["rating_pred"] = pred_results

In [9]:
from collections import defaultdict

# 각 사용자에 대한 추천 영화는 해당 사용자 아직 평가하지 않은 영화 중에서 예측값이 높은 순으로 한다
pred_user2items = defaultdict(list)
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
for user_id in movielens_train.user_id.unique():
    if user_id not in user_id2index:
        continue
    user_index = user_id2index[row["user_id"]]
    movie_indexes = np.argsort(-pred_matrix[user_index, :])
    for movie_index in movie_indexes:
        movie_id = user_movie_matrix.columns[movie_index]
        if movie_id not in user_evaluated_movies[user_id]:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break

pred_user2items

defaultdict(list,
            {139: [4993, 4226, 5952, 7153, 4306, 3996, 5349, 4963, 6539, 4886],
             149: [2571, 4226, 2762, 3996, 5349, 47, 2028, 4963, 6539, 4886],
             182: [3793, 47, 6874, 2329, 4011, 7361, 2706, 4878, 2502, 6333],
             215: [2329, 3147, 6711, 527, 2683, 3948, 8360, 5218, 6502, 1246],
             281: [2959, 2571, 4993, 3578, 4226, 5952, 2858, 2762, 7153, 4306],
             326: [2959, 2571, 4993, 3578, 4226, 5952, 2858, 2762, 7153, 4306],
             351: [2959, 4993, 3578, 4226, 5952, 2762, 7153, 4306, 3996, 3793],
             357: [2959, 4993, 3578, 4226, 5952, 2858, 2762, 7153, 4306, 3996],
             426: [2502,
              33794,
              7147,
              3949,
              5902,
              4848,
              33166,
              33493,
              2791,
              5995],
             456: [2959, 2571, 4993, 3578, 4226, 5952, 2858, 2762, 7153, 4306],
             459: [4886, 318, 2997, 4995, 4973, 50, 3897, 

In [10]:
# user_id=2인 사용자가 학습 데이터에서 평가를 부여한 영화 목록
movielens_train[movielens_train.user_id==2]

user_id  movie_id  rating  timestamp  \
4732        2       110     5.0  868245777   
5246        2       260     5.0  868244562   
5798        2       590     5.0  868245608   
6150        2       648     2.0  868244699   
6531        2       733     3.0  868244562   
6813        2       736     3.0  868244698   
7113        2       780     3.0  868244698   
7506        2       786     3.0  868244562   
7661        2       802     2.0  868244603   
7779        2       858     2.0  868245645   
8077        2      1049     3.0  868245920   
8127        2      1073     3.0  868244562   
8381        2      1210     4.0  868245644   
8771        2      1356     3.0  868244603   
9097        2      1544     3.0  868245920   

                                                  title  \
4732                                  Braveheart (1995)   
5246  Star Wars: Episode IV - A New Hope (a.k.a. Sta...   
5798                          Dances with Wolves (1990)   
6150                         Mission: Impossible (1996)   
6531                                   Rock, The (1996)   
6813                                     Twister (1996)   
7113               Independence Day (a.k.a. ID4) (1996)   
7506                                      Eraser (1996)   
7661                                  Phenomenon (1996)   
7779                              Godfather, The (1972)   
8077                 Ghost and the Darkness, The (1996)   
8127         Willy Wonka & the Chocolate Factory (1971)   
8381  Star Wars: Episode VI - Return of the Jedi (1983)   
8771                    Star Trek: First Contact (1996)   
9097  Lost World: Jurassic Park, The (Jurassic Park ...   

                                              genre  \
4732                           [Action, Drama, War]   
5246                    [Action, Adventure, Sci-Fi]   
5798                    [Adventure, Drama, Western]   
6150         [Action, Adventure, Mystery, Thriller]   
6531                  [Action, Adventure, Thriller]   
6813         [Action, Adventure, Romance, Thriller]   
7113               [Action, Adventure, Sci-Fi, War]   
7506                      [Action, Drama, Thriller]   
7661                               [Drama, Romance]   
7779                                 [Crime, Drama]   
8077                            [Action, Adventure]   
8127           [Children, Comedy, Fantasy, Musical]   
8381                    [Action, Adventure, Sci-Fi]   
8771          [Action, Adventure, Sci-Fi, Thriller]   
9097  [Action, Adventure, Horror, Sci-Fi, Thriller]   

                                                    tag  timestamp_rank  
4732  [bullshit history, medieval, bloodshed, hero, ...             8.0  
5246  [desert, quotable, lucas, gfei own it, seen mo...            17.0  
5798  [afi 100, lame, native, biopic, american india...            11.0  
6150  [confusing, confusing plot, memorable sequence...            12.0  
6531  [gfei own it, alcatraz, nicolas cage, sean con...            18.0  
6813  [disaster, disaster, storm, bill paxton, helen...            13.0  
7113  [action, alien invasion, aliens, will smith, a...            14.0  
7506  [arnold schwarzenegger, action, arnold, arnold...            19.0  
7661  [interesting concept, own, john travolta, john...            15.0  
7779  [oscar (best picture), marlon brando, classic,...             9.0  
8077  [lions, own, seen at the cinema, adventure, af...             6.0  
8127  [based on a book, roald dahl, based on a book,...            20.0  
8381  [desert, fantasy, sci-fi, space, lucas, gfei o...            10.0  
8771  [space, borg, futuristmovies.com, patrick stew...            16.0  
9097  [dinosaurs, dinosaurs, steven spielberg, borin...             7.0

In [11]:
pred_user2items[2]

[2959, 2571, 4993, 3578, 4226, 5952, 2858, 2762, 7153, 4306]

In [12]:
# user_id=2에 대한 추천(2959, 4993, 5952)
movies[movies.movie_id.isin([2959, 2571, 4993])]

movie_id                                              title  \
2487      2571                                 Matrix, The (1999)   
2874      2959                                  Fight Club (1999)   
4899      4993  Lord of the Rings: The Fellowship of the Ring,...   

                                 genre  \
2487        [Action, Sci-Fi, Thriller]   
2874  [Action, Crime, Drama, Thriller]   
4899      [Action, Adventure, Fantasy]   

                                                    tag  
2487  [artificial intelligence, hackers, heroine in ...  
2874  [based on a book, chuck palahniuk, edward nort...  
4899  [based on a book, big budget, new zealand, sce...